In [ ]:
#  ! recipes/ljspeech/xtts_v1/train_gpt_xtts.py

In [12]:
import os
import torch
import torchaudio
import IPython.display as ipd
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# Add here the xtts_config path
CONFIG_PATH = "recipes/ljspeech/xtts_v1/run/training/GPT_XTTS_LJSpeech_FT-November-02-2023_10+25AM-6fef4f90/config.json"
# Add here the vocab file that you have used to train the model
TOKENIZER_PATH = "recipes/ljspeech/xtts_v1/run/training/XTTS_v1.1_original_model_files/vocab.json"
# Add here the checkpoint that you want to do inference with
# XTTS_CHECKPOINT = "recipes/ljspeech/xtts_v1/run/training/XTTS_v1.1_original_model_files/model.pth"
XTTS_CHECKPOINT = "recipes/ljspeech/xtts_v1/run/training/GPT_XTTS_LJSpeech_FT-November-02-2023_10+25AM-6fef4f90/best_model.pth"

print("Loading model...")
config = XttsConfig()
config.load_json(CONFIG_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)

Loading model...


In [13]:
model.cpu()

Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Ide

In [20]:
# Add here the speaker reference
SPEAKER_REFERENCE = "prompts/dr-cut.webm"
gpt_cond_latent, diffusion_conditioning, speaker_embedding = model.get_conditioning_latents(audio_path=SPEAKER_REFERENCE)

LibsndfileError: Error opening 'prompts/dr-cut.webm': System error.

In [19]:
print("Inference...")
out = model.inference(
    "這裡是網際智慧的'語音生成'服務, 我是吳淡如，現在是人工智能生成的語音",
    "zh-cn",
    gpt_cond_latent,
    speaker_embedding,
    diffusion_conditioning,
    temperature=0.5, # Add custom parameters here
)
# torchaudio.save('xtts-ft.wav', torch.tensor(out["wav"]).unsqueeze(0), 24000)
ipd.Audio(torch.tensor(out["wav"]).unsqueeze(0), rate=24000) 

Inference...


In [1]:
#TOOL

In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

def remove_silence(wav_path, output_path, silence_thresh=-40, min_silence_len=300):
    # 载入音频文件
    sound = AudioSegment.from_wav(wav_path)
    
    # 寻找非静音的块
    nonsilent_chunks = detect_nonsilent(
        sound,
        min_silence_len=min_silence_len,  # 最小的静音长度，单位为毫秒
        silence_thresh=silence_thresh     # 小于该值的为静音，默认为-16dBFS
    )
    
    # 合并非静音块
    non_silent_audio = sum([sound[start:end] for start, end in nonsilent_chunks])
    
    # 导出结果
    non_silent_audio.export(output_path, format="wav")

# 使用函数去除空白部分
remove_silence("prompts/js-1.wav", "prompts/js-1-cut.wav")
